In [1]:
!pip install --upgrade --quiet  langchain langchain_experimental langchain-openai


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
! which python

'which' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip list

Package                  Version
------------------------ -----------
aiohappyeyeballs         2.6.1
aiohttp                  3.12.13
aiosignal                1.3.2
annotated-types          0.7.0
anyio                    4.9.0
asttokens                3.0.0
attrs                    25.3.0
certifi                  2025.6.15
charset-normalizer       3.4.2
colorama                 0.4.6
comm                     0.2.2
dataclasses-json         0.6.7
debugpy                  1.8.14
decorator                5.2.1
distro                   1.9.0
executing                2.2.0
frozenlist               1.7.0
greenlet                 3.2.3
h11                      0.16.0
httpcore                 1.0.9
httpx                    0.28.1
httpx-sse                0.4.0
idna                     3.10
ipykernel                6.29.5
ipython                  9.3.0
ipython_pygments_lexers  1.1.1
jedi                     0.19.2
jiter                    0.10.0
jsonpatch                1.33
jsonpointer         


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import AzureChatOpenAI


In [13]:
class TransactionRecord(BaseModel):
    transaction_id: str
    customer_id: str
    transaction_amount: float
    transaction_date: str
    payment_method: str
    product_category: str
    quantity: int
    customer_age: int
    customer_location: str
    device_used: str
    ip_address: str
    shipping_address: str
    billing_address: str
    is_fraudulent: int
    account_age_days: int
    transaction_hour: int

In [14]:
examples = [
    { "example": """transaction_id: 44e09e24-74b9-4041-9894-8734ad525faa, customer_id: 379, transaction_amount: 10.01, transaction_date: 10/01/2024 20:08, payment_method: debit card, product_category: home & garden, quantity: 4, customer_age: 37, customer_location: Christopherland, device_used: desktop, ip_address: 211.189.218.190, shipping_address: 471 Mary Courts Apt. 872\nMaryhaven, KS 16126, billing_address: 471 Mary Courts Apt. 872\nMaryhaven, KS 16126, is_fraudulent: 1, account_age_days: 30, transaction_hour: 20"""},
    {"example": """transaction_id: aa81b9ff-034d-4d8e-bec6-ed29b8ae5d54, customer_id: 19.25, transaction_amount: 11/02/2024 16:54, transaction_date: bank transfer, payment_method: clothing, product_category: 1, quantity: 32, customer_age: Marymouth, customer_location: mobile, device_used: 126.224.150.175, ip_address: 68763 Bryant Points Suite 057\nGuzmanfort, ME 02932, shipping_address: 139 Lopez Neck\nCarterview, DE 85571, billing_address: 0, is_fraudulent: 131, account_age_days: 16, transaction_hour: 16"""},
    {"example": """transaction_id: 7ac1febd-3e94-457e-bfe4-acf081dc8da0, customer_id: 104.07, transaction_amount: 07/01/2024 15:41, transaction_date: PayPal, payment_method: electronics, product_category: 1, quantity: 43, customer_age: Paynefurt, customer_location: tablet, device_used: 195.31.145.184, ip_address: Unit 5864 Box 9572\nDPO AE 69746, shipping_address: Unit 5864 Box 9572\nDPO AE 69746, billing_address: 0, is_fraudulent: 53, account_age_days: 15, transaction_hour: 15"""},
]

In [16]:
OPENAI_TEMPLATE = PromptTemplate(
    input_variables=["example"],
    template="{example}"
)



# Construct few-shot prompt
prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

# Create AzureChatOpenAI client
llm = AzureChatOpenAI(
    azure_endpoint="https://june16.openai.azure.com/",
    openai_api_key="59laGrhxCvqEGck89UoAwBVC0QURaxbbIybZAxLMSO0ndOm7FuHoJQQJ99BFACYeBjFXJ3w3AAAAACOGaizs",
    deployment_name="gpt-4.1",  # e.g., "gpt-35-turbo"
    openai_api_version="2025-01-01-preview",
    temperature=1
)

# Create synthetic data generator
subject = "TransactionRecord"
extra = "Include features like high-risk IP addresses, large amounts, and unrealistic timestamps."

for _ in range(5):
    prompt = prompt_template.format(subject=subject, extra=extra)
    response = llm.invoke(prompt)
    print(response.content)

transaction_id: c7f45d3b-1ddb-4b32-aa5c-83e512b217f3, customer_id: 528, transaction_amount: 1249.99, transaction_date: 13/32/2024 27:61, payment_method: bitcoin, product_category: luxury goods, quantity: 8, customer_age: 19, customer_location: East Ericamouth, device_used: smart fridge, ip_address: 203.0.113.45, shipping_address: 9458 Grant Estates Suite 578
Port Amberport, AZ 95413, billing_address: 3112 Smith Canyon
West Nicole, FL 55962, is_fraudulent: 1, account_age_days: 1, transaction_hour: 27

transaction_id: 0f88199e-6f3a-4854-af6f-55e6c519b4d9, customer_id: 004, transaction_amount: 0.01, transaction_date: 02/30/2024 00:00, payment_method: credit card, product_category: gaming, quantity: 100, customer_age: 91, customer_location: Unknownville, device_used: mobile, ip_address: 198.51.100.99, shipping_address: 123 Fraud Lane
Shadytown, NV 00000, billing_address: 123 Fraud Lane
Shadytown, NV 00000, is_fraudulent: 1, account_age_days: 0, transaction_hour: 00

transaction_id: 5e93e8b